In [10]:
import torch
import numpy as np
from pathlib import Path
from utils.general import non_max_suppression, scale_boxes, check_img_size, Profile
from utils.segment.general import process_mask, process_mask_native
from utils.augmentations import letterbox
from models.common import DetectMultiBackend
from utils.dataloaders import LoadImages
import cv2
import tifffile as tiff

def process_mask(proto, det, shape, upsample=True):
    # Ensure det is correctly reshaped for matrix multiplication
    masks = proto @ det.T  # Matrix multiplication
    masks = masks.sigmoid()
    if upsample:
        masks = torch.nn.functional.interpolate(masks.unsqueeze(0), size=shape, mode='bilinear', align_corners=False).squeeze(0)
    return masks

# Define variables
model_path_tif = '/datadisk2/c241_ml02/workspace/yolov5/runs/train-seg/exp31/weights/best.pt'  # Path to the model
model_path_rgb = '/datadisk2/c241_ml02/workspace/vanilla_yolo/yolov5/runs/train-seg/exp6/weights/best.pt'
image_path_tif = '/datadisk2/c241_ml02/workspace/train_data/20ch_rescaled/fold_1/train/images/train_s2_image_0.tif'  # Path to the image
image_path_rgb = '/datadisk2/c241_ml02/workspace/preprocessed_data/1_2_3_resized/images/train_s2_image_0.jpg'

image_path = image_path_tif
model_path = model_path_tif

if image_path.endswith('.tif') :
    height, width, channel = tiff.imread(image_path).shape
elif image_path.endswith('.jpg') or image_path.endswith('jpeg') :
    height, width, channel = cv2.imread(image_path).shape

conf_thres = 0.25  # Confidence threshold
iou_thres = 0.45  # IoU threshold for NMS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available
dt = Profile(device=device), Profile(device=device), Profile(device=device)

# Load model
imgsz=256
model = DetectMultiBackend(model_path, device=device, dnn=False, fp16=False)
stride, pt, jit, engine = model.stride, model.pt, model.jit, model.engine

model.eval()
model.warmup(imgsz=(1 if pt else batch_size, 20, imgsz, imgsz))

# Load image
dataset = LoadImages(image_path, img_size=640, stride=stride, auto=pt)
for path, img, im0s, vid_cap, s in dataset:
    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    # img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if len(img.shape) == 3:
        img = img[None]  # expand for batch dim

    # Inference 
    with dt[1]:
        preds, protos, train_out = model(img)

    # NMS

    with dt[2]:
        preds = non_max_suppression(
            preds, conf_thres, iou_thres, labels=None, multi_label=False, agnostic=True, max_det=150, nm=32
        ) 

    # Process predictions
    masks = []  # Array to hold mask arrays
    im0 = im0s.copy()
    for si, (pred, proto) in enumerate(zip(preds, protos)):
        if len(pred):
            # Rescale boxes from img_size to im0 size
            pred[:, :4] = scale_boxes(img.shape[2:], pred[:, :4], im0.shape).round()
            # Extract masks
            pred_masks = process_mask(proto, pred, pred[:, :4], (640, 640))
            masks.append(pred_masks)

# 'masks' now contains the mask arrays for the image


Fusing layers... 
YOLOv5m-seg summary: 220 layers, 21681734 parameters, 0 gradients, 75.8 GFLOPs


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5120x160 and 38x12)

In [28]:
validation_path = '/datadisk2/c241_ml02/workspace/yolov5/segment/val.py'
model_path = '/datadisk2/c241_ml02/workspace/yolov5/runs/train-seg/exp33/weights/best.pt'

data_path = '/datadisk2/c241_ml02/workspace/train_data/20ch_rescaled/configs/fold_1.yaml'

!python {validation_path} --weights {model_path} --data {data_path} --conf-thres 0.45

segment/val: data=/datadisk2/c241_ml02/workspace/train_data/20ch_rescaled/configs/fold_1.yaml, weights=['/datadisk2/c241_ml02/workspace/yolov5/runs/train-seg/exp33/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.45, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val-seg, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.45 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-313-g712de55a Python-3.12.3 torch-2.3.0.post300 CUDA:0 (NVIDIA H100 80GB HBM3, 81004MiB)

Fusing layers... 
YOLOv5m-seg summary: 220 layers, 21681734 parameters, 0 gradients, 75.8 GFLOPs
val: Scanning /datadisk2/c241_ml02/workspace/train_data/20ch_rescaled/fold_1/val
                 Class     Images  Instances      Box(P          R      mAP50  mWARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances      Box(P

In [ ]:
print(proto)